In [1]:
import argparse
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torchvision import datasets, transforms
from torch.autograd import Variable
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn.metrics import accuracy_score
import tqdm

모델 구조 불러오기

In [2]:
from src.lib import *

공격 준비를 위한 transform, dataset, loader 설정

In [3]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

# define transforms
transform = transforms.Compose([
        transforms.ToTensor(),
        normalize
])

dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
train_set, valid_set= torch.utils.data.random_split(dataset, [40000,10000])
train_loader = torch.utils.data.DataLoader(train_set, batch_size=64)
valid_loader = torch.utils.data.DataLoader(valid_set, batch_size=64, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)


Files already downloaded and verified
Files already downloaded and verified


In [4]:
from src.lib import *

NOH

Dataset ; CIFAR10

Victim ; MobileNet

Shadow ; SqueezeNet(Untrained)

In [6]:
clientMobilenet = ClientMobileNet()
serverMobilenet = ServerMobileNet()
clientMobilenet.load_state_dict(torch.load('./assets/mobilenet_cifar10_c0.pth'))
serverMobilenet.load_state_dict(torch.load('./assets/mobilenet_cifar10_server.pth'))

<All keys matched successfully>

In [7]:
num_epoch = 50
shadow = SqueezeNet()
optimizer = optim.Adam(shadow.parameters())
criterion = nn.CrossEntropyLoss()
device = 'cpu'
shadow.train()
softmax = nn.Softmax(dim=1)
for epoch in range(num_epoch):
    total_loss = 0
    for x,y in test_loader:
        y_label = softmax(serverMobilenet(clientMobilenet(x)).detach())
        y_shadow = shadow(x)
        optimizer.zero_grad()
        loss = criterion(y_shadow, y_label)
        total_loss += loss
        loss.backward()
        optimizer.step()
    print(f"epoch {(epoch+1)}: loss {total_loss}")
print("done")
    

epoch 1: loss 344.3273010253906
epoch 2: loss 330.463623046875
epoch 3: loss 324.4977111816406
epoch 4: loss 320.3070068359375
epoch 5: loss 316.1141052246094
epoch 6: loss 312.96038818359375
epoch 7: loss 310.5359191894531
epoch 8: loss 308.1898498535156
epoch 9: loss 306.33953857421875
epoch 10: loss 305.0871276855469
epoch 11: loss 303.06329345703125
epoch 12: loss 300.9832763671875
epoch 13: loss 303.3331298828125
epoch 14: loss 299.9398193359375
epoch 15: loss 298.787353515625
epoch 16: loss 295.6677551269531
epoch 17: loss 293.6312561035156
epoch 18: loss 291.9891052246094
epoch 19: loss 289.7529296875
epoch 20: loss 288.3203430175781
epoch 21: loss 285.7350158691406
epoch 22: loss 285.3455505371094
epoch 23: loss 285.288818359375
epoch 24: loss 282.95635986328125
epoch 25: loss 280.7430725097656
epoch 26: loss 279.1961364746094
epoch 27: loss 277.8844299316406
epoch 28: loss 276.6181335449219
epoch 29: loss 276.6001281738281
epoch 30: loss 275.6732482910156
epoch 31: loss 277.24

In [9]:
torch.save(shadow,'mobile_cifar10_whole_squeezenet.pth')

In [1]:
shadow.eval()
acc, tot = 0, 0
for x, y in valid_loader:
    y_pred = shadow(x)
    acc += (y==y_pred.argmax(1)).sum()
    tot += len(y)
print(f"test acc : {acc}/{tot}")


test acc : 5252/10000


NOH

Dataset; CIFAR10

Victim; MobileNet

Shadow; MobileNet(Untrained)

In [9]:
num_epoch = 50
shadow = MobileNet()
optimizer = optim.Adam(shadow.parameters())
criterion = nn.CrossEntropyLoss()
device = 'cpu'
shadow.train()
softmax = nn.Softmax(dim=1)
for epoch in range(num_epoch):
    total_loss = 0
    for x,y in test_loader:
        y_label = softmax(serverMobilenet(clientMobilenet(x)).detach())
        y_shadow = shadow(x)
        optimizer.zero_grad()
        loss = criterion(y_shadow, y_label)
        total_loss += loss
        loss.backward()
        optimizer.step()
    print(f"epoch {(epoch+1)}: loss {total_loss}")
print("done")

epoch 1: loss 309.802978515625
epoch 2: loss 258.3976135253906
epoch 3: loss 226.8338623046875
epoch 4: loss 202.29734802246094
epoch 5: loss 186.60231018066406
epoch 6: loss 175.2358856201172
epoch 7: loss 162.50765991210938
epoch 8: loss 152.2112579345703
epoch 9: loss 144.7798614501953
epoch 10: loss 140.2115936279297
epoch 11: loss 135.78221130371094
epoch 12: loss 132.74884033203125
epoch 13: loss 130.4856414794922
epoch 14: loss 129.74569702148438
epoch 15: loss 126.74622344970703
epoch 16: loss 124.76313018798828
epoch 17: loss 123.48863220214844
epoch 18: loss 122.2680892944336
epoch 19: loss 122.30835723876953
epoch 20: loss 121.899658203125
epoch 21: loss 121.6991195678711
epoch 22: loss 120.63117980957031
epoch 23: loss 119.20167541503906
epoch 24: loss 118.8907470703125
epoch 25: loss 118.72772979736328
epoch 26: loss 118.56607055664062
epoch 27: loss 118.7629623413086
epoch 28: loss 118.95840454101562
epoch 29: loss 118.7726821899414
epoch 30: loss 118.88065338134766
epoch

In [2]:
shadow.eval()
acc, tot = 0, 0
for x, y in valid_loader:
    y_pred = shadow(x)
    acc += (y==y_pred.argmax(1)).sum()
    tot += len(y)
print(f"test acc : {acc}/{tot}")

test acc : 5851/10000


In [11]:
torch.save(shadow,'mobile_cifar10_whole_mobilenet.pth')

In [8]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

# define transforms
transform = transforms.Compose([
        transforms.ToTensor(),
        normalize
])

dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
train_set, valid_set= torch.utils.data.random_split(dataset, [40000,10000])
train_loader = torch.utils.data.DataLoader(train_set, batch_size=64)
valid_loader = torch.utils.data.DataLoader(valid_set, batch_size=64, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [26]:
from src.lib import *

digit5학습 모델 공격

In [6]:
import os
import time
import sys

from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.utils.data import Subset
from torchvision.datasets import MNIST, SVHN, USPS

from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import copy

In [7]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)),
                                transforms.Resize((32, 32))
                               ])
transform_to_rgb = transforms.Compose([transforms.Grayscale(num_output_channels=3),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)),
                                       transforms.Resize((32, 32))
                                      ])

In [8]:
import torchvision
train_usps = torchvision.datasets.USPS(root='./data' + 'usps_data', train=True, download=True, transform=transform_to_rgb)
test_usps = torchvision.datasets.USPS(root='./data' + 'usps_data', train=False, download=True, transform=transform_to_rgb)

train_mnist = torchvision.datasets.MNIST(root='./data' + 'mnist_data', train=True, download=True, transform=transform_to_rgb)
test_mnist = torchvision.datasets.MNIST(root='./data' + 'mnist_data', train=False, download=True, transform=transform_to_rgb)

In [5]:
train_loader = torch.utils.data.DataLoader(train_usps, batch_size=64, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_usps, batch_size=64, shuffle=False)

In [8]:
clientMobilenet = ClientMobileNet()
serverMobilenet = ServerMobileNet()
clientMobilenet.load_state_dict(torch.load('./assets/mobilenet_digit5_c0.pth'))
serverMobilenet.load_state_dict(torch.load('./assets/mobilenet_digit5_server.pth'))

<All keys matched successfully>

Target2-1 : Different Architecture
Blank wholesome blackbox model
same domain(USPS)

In [8]:
num_epoch = 50
shadow = SqueezeNet(num_classes=10)
optimizer = optim.Adam(shadow.parameters())
criterion = nn.CrossEntropyLoss()
device = 'cpu'
shadow.train()
softmax = nn.Softmax(dim=1)
for epoch in range(num_epoch):
    total_loss = 0
    for x,y in train_loader:
        y_label = softmax(serverMobilenet(clientMobilenet(x)).detach())
        y_shadow = shadow(x)
        optimizer.zero_grad()
        loss = criterion(y_shadow, y_label)
        total_loss += loss
        loss.backward()
        optimizer.step()
    print(f"epoch {(epoch+1)}: loss {total_loss}")
print("done")

epoch 1: loss 229.4803924560547
epoch 2: loss 200.88006591796875
epoch 3: loss 189.63145446777344
epoch 4: loss 186.33233642578125
epoch 5: loss 182.88233947753906
epoch 6: loss 184.66497802734375
epoch 7: loss 179.89984130859375
epoch 8: loss 177.9769744873047
epoch 9: loss 178.9838409423828
epoch 10: loss 177.656005859375
epoch 11: loss 177.15164184570312
epoch 12: loss 177.6970977783203
epoch 13: loss 178.11175537109375
epoch 14: loss 176.59713745117188
epoch 15: loss 177.08155822753906
epoch 16: loss 176.92271423339844
epoch 17: loss 175.97311401367188
epoch 18: loss 176.08445739746094
epoch 19: loss 176.03648376464844
epoch 20: loss 174.8348388671875
epoch 21: loss 174.2608642578125
epoch 22: loss 174.98480224609375
epoch 23: loss 174.57888793945312
epoch 24: loss 173.9051055908203
epoch 25: loss 173.39186096191406
epoch 26: loss 173.59771728515625
epoch 27: loss 173.7978057861328
epoch 28: loss 172.5680389404297
epoch 29: loss 172.8959197998047
epoch 30: loss 173.48202514648438
e

In [9]:
shadow.eval()
acc, tot = 0, 0
for x, y in test_loader:
    y_pred = shadow(x)
    acc += (y==y_pred.argmax(1)).sum()
    tot += len(y)
print(f"test acc : {acc}/{tot}")

test acc : 799/2007


In [10]:
torch.save(shadow,'mobile_usps_whole_squeezenet.pth')

In [9]:
# train 8000
indices = range(8000,16000)
mnist_train_loader = torch.utils.data.DataLoader(Subset(train_mnist, indices), batch_size=64, shuffle=False)
mnist_test_loader = torch.utils.data.DataLoader(test_mnist, batch_size=64, shuffle=False)
train_loader = torch.utils.data.DataLoader(train_mnist, batch_size=64, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_mnist, batch_size=64, shuffle=False)

Target: Mobilenet-Digit5
Attacker : Squeezenet MNIST(same dataset)

In [28]:
num_epoch = 20
shadow = SqueezeNet(num_classes=10)
optimizer = optim.Adam(shadow.parameters())
criterion = nn.CrossEntropyLoss()
device = 'cpu'
batch=64
shadow.train()
softmax = nn.Softmax(dim=1)
for epoch in range(num_epoch):
    total_loss = 0
    for x,y in mnist_train_loader:
        y_label = softmax(serverMobilenet(clientMobilenet(x)).detach())
        y_shadow = shadow(x)
        optimizer.zero_grad()
        loss = criterion(y_shadow, y_label)
        total_loss += loss
        loss.backward()
        optimizer.step()
    print(f"epoch {(epoch+1)}: loss {total_loss/batch}")
print("done")

epoch 1: loss 22.8120174407959
epoch 2: loss 21.09133529663086
epoch 3: loss 20.828954696655273
epoch 4: loss 20.78696632385254
epoch 5: loss 20.70159339904785
epoch 6: loss 20.680931091308594
epoch 7: loss 20.603588104248047
epoch 8: loss 20.59605598449707
epoch 9: loss 20.589523315429688
epoch 10: loss 20.557926177978516
epoch 11: loss 20.57498550415039
epoch 12: loss 20.540122985839844
epoch 13: loss 20.55400848388672
epoch 14: loss 20.476640701293945
epoch 15: loss 20.494062423706055
epoch 16: loss 20.47201156616211
epoch 17: loss 20.410049438476562
epoch 18: loss 20.42901039123535
epoch 19: loss 20.39900779724121
epoch 20: loss 20.35149383544922
done


In [29]:
shadow.eval()
acc, tot = 0, 0
for x, y in test_loader:
    y_pred = shadow(x)
    acc += (y==y_pred.argmax(1)).sum()
    tot += len(y)
print(f"test acc : {acc}/{tot}")

test acc : 5044/10000


In [31]:
torch.save(shadow,'mobile_mnist_whole_squeezenet.pth')

In [8]:
train_loader = torch.utils.data.DataLoader(train_usps, batch_size=64, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_usps, batch_size=64, shuffle=False)

Target2-2 : Different Architecture
Blank wholesome blackbox model
different domain(Noise)

In [16]:
num_epoch = 20
shadow = SqueezeNet(num_classes=10)
optimizer = optim.Adam(shadow.parameters())
criterion = nn.CrossEntropyLoss()
device = 'cpu'
batch=64
shadow.train()
softmax = nn.Softmax(dim=1)
for epoch in range(num_epoch):
    total_loss = 0
    for x,y in train_loader:
        x = torch.randn_like(x)
        y_label = softmax(serverMobilenet(clientMobilenet(x)).detach())
        y_shadow = shadow(x)
        optimizer.zero_grad()
        loss = criterion(y_shadow, y_label)
        total_loss += loss
        loss.backward()
        optimizer.step()
    print(f"epoch {(epoch+1)}: loss {total_loss/batch}")
print("done")

epoch 1: loss 261.7731018066406
epoch 2: loss 261.4520263671875
epoch 3: loss 261.480224609375
epoch 4: loss 261.5951843261719
epoch 5: loss 261.60589599609375
epoch 6: loss 261.54962158203125
epoch 7: loss 261.5716247558594
epoch 8: loss 261.5108642578125
epoch 9: loss 261.626708984375
epoch 10: loss 261.5336608886719
epoch 11: loss 261.6938171386719
epoch 12: loss 261.50103759765625
epoch 13: loss 261.59173583984375
epoch 14: loss 261.42791748046875
epoch 15: loss 261.4223327636719
epoch 16: loss 261.43798828125
epoch 17: loss 261.34271240234375
epoch 18: loss 261.3768005371094
epoch 19: loss 261.6382751464844
epoch 20: loss 261.51336669921875
done


In [18]:
shadow.eval()
acc, tot = 0, 0
for x, y in test_loader:
    y_pred = shadow(x)
    acc += (y==y_pred.argmax(1)).sum()
    tot += len(y)
print(f"test acc : {acc}/{tot}")

test acc : 198/2007


In [19]:
torch.save(shadow,'mobile_usps_whole_squeezenet_noise.pth')

In [8]:
clientMobilenet = ClientMobileNet()
serverMobilenet = ServerMobileNet()
clientMobilenet.load_state_dict(torch.load('./assets/mobilenet_digit5_c0.pth'))
serverMobilenet.load_state_dict(torch.load('./assets/mobilenet_digit5_server.pth'))

<All keys matched successfully>

Target2-3 : Same Architecture
Blank wholesome blackbox model
same domain(MNIST)

In [10]:
# train 8000
indices = range(8000,16000)
mnist_train_loader = torch.utils.data.DataLoader(Subset(train_mnist, indices), batch_size=64, shuffle=False)
mnist_test_loader = torch.utils.data.DataLoader(test_mnist, batch_size=64, shuffle=False)
train_loader = torch.utils.data.DataLoader(train_mnist, batch_size=64, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_mnist, batch_size=64, shuffle=False)

In [12]:
num_epoch = 20
shadow = MobileNet()
optimizer = optim.Adam(shadow.parameters())
criterion = nn.CrossEntropyLoss()
device = 'cpu'
batch=64
shadow.train()
softmax = nn.Softmax(dim=1)
for epoch in range(num_epoch):
    total_loss = 0
    for x,y in mnist_train_loader:
        y_label = softmax(serverMobilenet(clientMobilenet(x)).detach())
        y_shadow = shadow(x)
        optimizer.zero_grad()
        loss = criterion(y_shadow, y_label)
        total_loss += loss
        loss.backward()
        optimizer.step()
    print(f"epoch {(epoch+1)}: loss {total_loss/batch}")
print("done")

epoch 1: loss 1.824452519416809
epoch 2: loss 0.5201795697212219
epoch 3: loss 0.27422887086868286
epoch 4: loss 0.2657982110977173
epoch 5: loss 0.23205013573169708
epoch 6: loss 0.22181165218353271
epoch 7: loss 0.20782308280467987
epoch 8: loss 0.19835729897022247
epoch 9: loss 0.18525664508342743
epoch 10: loss 0.1815684586763382
epoch 11: loss 0.17734026908874512
epoch 12: loss 0.17177267372608185
epoch 13: loss 0.17308466136455536
epoch 14: loss 0.1702556163072586
epoch 15: loss 0.1667814701795578
epoch 16: loss 0.1675107628107071
epoch 17: loss 0.15633940696716309
epoch 18: loss 0.15821707248687744
epoch 19: loss 0.15416815876960754
epoch 20: loss 0.16101135313510895
done


In [13]:
shadow.eval()
acc, tot = 0, 0
for x, y in test_loader:
    y_pred = shadow(x)
    acc += (y==y_pred.argmax(1)).sum()
    tot += len(y)
print(f"test acc : {acc}/{tot}")

test acc : 9721/10000


In [14]:
torch.save(shadow,'mobile_mnist_whole_mobile.pth')

Target2-3 : Same Architecture
Blank wholesome blackbox model
same domain(USPS)

In [21]:
num_epoch = 20
shadow = MobileNet()
optimizer = optim.Adam(shadow.parameters())
criterion = nn.CrossEntropyLoss()
device = 'cpu'
batch=64
shadow.train()
softmax = nn.Softmax(dim=1)
for epoch in range(num_epoch):
    total_loss = 0
    for x,y in train_loader:
        y_label = softmax(serverMobilenet(clientMobilenet(x)).detach())
        y_shadow = shadow(x)
        optimizer.zero_grad()
        loss = criterion(y_shadow, y_label)
        total_loss += loss
        loss.backward()
        optimizer.step()
    print(f"epoch {(epoch+1)}: loss {total_loss/batch}")
print("done")

epoch 1: loss 2.040894031524658
epoch 2: loss 1.0008713006973267
epoch 3: loss 0.7710716724395752
epoch 4: loss 0.6812163591384888
epoch 5: loss 0.6324265003204346
epoch 6: loss 0.5934742093086243
epoch 7: loss 0.5745225548744202
epoch 8: loss 0.5666912198066711
epoch 9: loss 0.5416908860206604
epoch 10: loss 0.5468255877494812
epoch 11: loss 0.5244064927101135
epoch 12: loss 0.5182403922080994
epoch 13: loss 0.5096077919006348
epoch 14: loss 0.5216582417488098
epoch 15: loss 0.49690723419189453
epoch 16: loss 0.49417728185653687
epoch 17: loss 0.4856204688549042
epoch 18: loss 0.48820456862449646
epoch 19: loss 0.47295477986335754
epoch 20: loss 0.46843668818473816
done


In [22]:
shadow.eval()
acc, tot = 0, 0
for x, y in test_loader:
    y_pred = shadow(x)
    acc += (y==y_pred.argmax(1)).sum()
    tot += len(y)
print(f"test acc : {acc}/{tot}")

test acc : 1781/2007


In [23]:
torch.save(shadow,'mobile_usps_whole_mobilenet_noise_ep20.pth')

Target2-1 : Same Architecture
Blank wholesome blackbox model
Different domain(Noise)

In [28]:
num_epoch = 2
shadow = MobileNet()
optimizer = optim.Adam(shadow.parameters())
criterion = nn.CrossEntropyLoss()
device = 'cpu'
batch=64
shadow.train()
softmax = nn.Softmax(dim=1)
for epoch in range(num_epoch):
    total_loss = 0
    for x,y in train_loader:
        x = torch.randn_like(x)
        y_label = softmax(serverMobilenet(clientMobilenet(x)).detach())
        y_shadow = shadow(x)
        optimizer.zero_grad()
        loss = criterion(y_shadow, y_label)
        total_loss += loss
        loss.backward()
        optimizer.step()
    print(f"epoch {(epoch+1)}: loss {total_loss/batch}")
print("done")

epoch 1: loss 22.642337799072266
epoch 2: loss 22.539932250976562
done


Noise는 학습 불가한것으로 결론..

Target2-1 : Same Architecture
Partial blackbox model. same front attached
same domain(usps)

In [15]:
shadow_front = ClientMobileNet()
shadow_front.load_state_dict(torch.load('./assets/mobilenet_digit5_c0.pth'))
shadow = ServerMobileNet()

In [16]:
num_epoch = 20
optimizer = optim.Adam(shadow.parameters())
criterion = nn.CrossEntropyLoss()
device = 'cpu'
batch=64
shadow.train()
softmax = nn.Softmax(dim=1)
for epoch in range(num_epoch):
    total_loss = 0
    for x,y in train_loader:
        y_label = softmax(serverMobilenet(clientMobilenet(x)).detach())
        y_shadow = softmax(shadow(shadow_front(x)))
        optimizer.zero_grad()
        loss = criterion(y_shadow, y_label)
        total_loss += loss
        loss.backward()
        optimizer.step()
    print(f"epoch {(epoch+1)}: loss {total_loss/batch}")
print("done")

epoch 1: loss 3.4655168056488037
epoch 2: loss 3.0560548305511475
epoch 3: loss 2.9641506671905518
epoch 4: loss 2.9444713592529297
epoch 5: loss 2.907142162322998
epoch 6: loss 2.8728983402252197
epoch 7: loss 2.8702809810638428
epoch 8: loss 2.8783111572265625
epoch 9: loss 2.8668599128723145
epoch 10: loss 2.8537611961364746
epoch 11: loss 2.851823091506958
epoch 12: loss 2.868387460708618
epoch 13: loss 2.8465332984924316
epoch 14: loss 2.8660573959350586
epoch 15: loss 2.8559341430664062
epoch 16: loss 2.833228349685669
epoch 17: loss 2.840780019760132
epoch 18: loss 2.8572661876678467
epoch 19: loss 2.838947057723999
epoch 20: loss 2.8413050174713135
done


In [18]:
shadow.eval()
acc, tot = 0, 0
for x, y in test_loader:
    y_pred = shadow(shadow_front(x))
    acc += (y==y_pred.argmax(1)).sum()
    tot += len(y)
print(f"test acc : {acc}/{tot}")

test acc : 1788/2007


Target2-1 : Same Architecture
Partial blackbox model. Different front attached
Different domain(usps)

In [21]:
shadow_front = ClientMobileNet()
shadow_front.load_state_dict(torch.load('./assets/mobilenet_digit5_c1.pth'))
shadow = ServerMobileNet()

In [22]:
num_epoch = 20
optimizer = optim.Adam(shadow.parameters())
criterion = nn.CrossEntropyLoss()
device = 'cpu'
batch=64
shadow.train()
softmax = nn.Softmax(dim=1)
for epoch in range(num_epoch):
    total_loss = 0
    for x,y in train_loader:
        y_label = softmax(serverMobilenet(clientMobilenet(x)).detach())
        y_shadow = softmax(shadow(shadow_front(x)))
        optimizer.zero_grad()
        loss = criterion(y_shadow, y_label)
        total_loss += loss
        loss.backward()
        optimizer.step()
    print(f"epoch {(epoch+1)}: loss {total_loss/batch}")
print("done")

epoch 1: loss 3.4753286838531494
epoch 2: loss 3.1104323863983154
epoch 3: loss 3.007295846939087
epoch 4: loss 2.9293928146362305
epoch 5: loss 2.88307785987854
epoch 6: loss 2.8843109607696533
epoch 7: loss 2.8699681758880615
epoch 8: loss 2.8695759773254395
epoch 9: loss 2.866029739379883
epoch 10: loss 2.8651697635650635
epoch 11: loss 2.8453316688537598
epoch 12: loss 2.852083206176758
epoch 13: loss 2.868345022201538
epoch 14: loss 2.8736839294433594
epoch 15: loss 2.834691047668457
epoch 16: loss 2.855942487716675
epoch 17: loss 2.8589773178100586
epoch 18: loss 2.842564821243286
epoch 19: loss 2.839914321899414
epoch 20: loss 2.8349411487579346
done


In [23]:
shadow.eval()
acc, tot = 0, 0
for x, y in test_loader:
    y_pred = shadow(shadow_front(x))
    acc += (y==y_pred.argmax(1)).sum()
    tot += len(y)
print(f"test acc : {acc}/{tot}")

test acc : 1806/2007


In [24]:
torch.save(shadow,'mobile_usps_partial_mobilenet_ep20.pth')

Target2-1 : Same Architecture
Partial blackbox model. Different front attached(c1)
same domain(MNIST)

In [15]:
shadow_front = ClientMobileNet()
shadow_front.load_state_dict(torch.load('./assets/mobilenet_digit5_c1.pth'))
shadow = ServerMobileNet()

In [9]:
# train 8000
indices = range(8000,16000)
mnist_train_loader = torch.utils.data.DataLoader(Subset(train_mnist, indices), batch_size=64, shuffle=False)
mnist_test_loader = torch.utils.data.DataLoader(test_mnist, batch_size=64, shuffle=False)
train_loader = torch.utils.data.DataLoader(train_mnist, batch_size=64, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_mnist, batch_size=64, shuffle=False)

In [17]:
num_epoch = 20
optimizer = optim.Adam(shadow.parameters())
criterion = nn.CrossEntropyLoss()
device = 'cpu'
batch=64
shadow.train()
softmax = nn.Softmax(dim=1)
for epoch in range(num_epoch):
    total_loss = 0
    for x,y in mnist_train_loader:
        y_label = softmax(serverMobilenet(clientMobilenet(x)).detach())
        y_shadow = softmax(shadow(shadow_front(x)))
        optimizer.zero_grad()
        loss = criterion(y_shadow, y_label)
        total_loss += loss
        loss.backward()
        optimizer.step()
    print(f"epoch {(epoch+1)}: loss {total_loss/batch}")
print("done")

epoch 1: loss 3.746610641479492
epoch 2: loss 3.2019877433776855
epoch 3: loss 3.0356059074401855
epoch 4: loss 2.988276720046997
epoch 5: loss 2.9874203205108643
epoch 6: loss 2.957688808441162
epoch 7: loss 2.9688665866851807
epoch 8: loss 2.9690699577331543
epoch 9: loss 2.940988540649414
epoch 10: loss 2.9425668716430664
epoch 11: loss 2.939539670944214
epoch 12: loss 2.9411840438842773
epoch 13: loss 2.95296573638916
epoch 14: loss 2.9514353275299072
epoch 15: loss 2.949476957321167
epoch 16: loss 2.9341206550598145
epoch 17: loss 2.941539764404297
epoch 18: loss 2.9480156898498535
epoch 19: loss 2.9300954341888428
epoch 20: loss 2.931401014328003
done


In [18]:
shadow.eval()
acc, tot = 0, 0
for x, y in test_loader:
    y_pred = shadow(shadow_front(x))
    acc += (y==y_pred.argmax(1)).sum()
    tot += len(y)
print(f"test acc : {acc}/{tot}")

test acc : 9528/10000


In [19]:
torch.save(shadow,'mobile_mnist_c1_partial_mobilenet_ep20.pth')

NOH

Dataset; CIFAR10

Target; MobileNet

Shadow; Mobile Front(pretrained) + SqueezeNet back


In [3]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

# define transforms
transform = transforms.Compose([
        transforms.ToTensor(),
        normalize
])

dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data',train=False, download=True, transform=transform)
train_set, valid_set= torch.utils.data.random_split(dataset, [40000,10000])
train_loader = torch.utils.data.DataLoader(train_set, batch_size=64)
valid_loader = torch.utils.data.DataLoader(valid_set, batch_size=64, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
from src.lib import *

In [4]:
clientMobilenet = ClientMobileNet()
serverMobilenet = ServerMobileNet()
clientMobilenet.load_state_dict(torch.load('./assets/mobilenet_cifar10_c0.pth'))
serverMobilenet.load_state_dict(torch.load('./assets/mobilenet_cifar10_server.pth'))

<All keys matched successfully>

In [5]:
shadow_client = ClientMobileNet()
shadow_client.load_state_dict(torch.load('./assets/mobilenet_cifar10_c0.pth'))
shadow_server = ShadowSqueezeNet()
mobile_squeeze = shadow_server.load_front_model(shadow_client)

In [8]:
num_epoch = 50
optimizer = optim.Adam(shadow_server.parameters())
criterion = nn.CrossEntropyLoss()
device = 'cpu'
batch=64
shadow_server.train()
print(1)
softmax = nn.Softmax(dim=1)
for epoch in range(num_epoch):
    total_loss = 0
    for x,y in test_loader:
        y_label = softmax(serverMobilenet(clientMobilenet(x)).detach())
        y_shadow = shadow_server(x)
        optimizer.zero_grad()
        loss = criterion(y_shadow, y_label)
        total_loss += loss
        loss.backward()
        optimizer.step()
    print(f"epoch {(epoch+1)}: loss {total_loss/batch}")
print("done")

1
epoch 1: loss 5.235965251922607
epoch 2: loss 4.825255393981934
epoch 3: loss 4.582886695861816
epoch 4: loss 4.393383979797363
epoch 5: loss 4.286125183105469
epoch 6: loss 4.16935920715332
epoch 7: loss 4.129035949707031
epoch 8: loss 4.025527000427246
epoch 9: loss 3.9936635494232178
epoch 10: loss 3.8933160305023193
epoch 11: loss 3.817272424697876
epoch 12: loss 3.7791669368743896
epoch 13: loss 3.7239251136779785
epoch 14: loss 3.6836495399475098
epoch 15: loss 3.6379024982452393
epoch 16: loss 3.6065304279327393
epoch 17: loss 3.583343267440796
epoch 18: loss 3.571096658706665
epoch 19: loss 3.5324909687042236
epoch 20: loss 3.4845147132873535
epoch 21: loss 3.506573438644409
epoch 22: loss 3.491093158721924
epoch 23: loss 3.4377591609954834
epoch 24: loss 3.4294557571411133
epoch 25: loss 3.3747241497039795
epoch 26: loss 3.371398687362671
epoch 27: loss 3.360705614089966
epoch 28: loss 3.4029669761657715
epoch 29: loss 3.3610198497772217
epoch 30: loss 3.3191468715667725
epo

In [11]:
shadow_server.eval()
acc, tot = 0, 0
for x, y in valid_loader:
    y_pred = shadow_server(x)
    acc += (y==y_pred.argmax(1)).sum()
    tot += len(y)
print(f"test acc : {acc}/{tot}")

test acc : 5433/10000


In [10]:
torch.save(shadow_server, './mobile_squeeze_cifar10_co.pth')

In [12]:
clientSqueezenet = ClientSqueezeNet()
serverSqueezenet = ServerSqueezeNet()
clientSqueezenet.load_state_dict(torch.load('./assets/squeezenet_cifar10_c0.pth'))
serverSqueezenet.load_state_dict(torch.load('./assets/squeezenet_cifar10_server.pth'))

<All keys matched successfully>

In [13]:
shadow_client = ClientSqueezeNet()
shadow_client.load_state_dict(torch.load('./assets/squeezenet_cifar10_c0.pth'))
shadow_server = ShadowMobileNet()
squeeze_mobile = shadow_server.load_front_model(shadow_client)

In [14]:
num_epoch = 20
optimizer = optim.Adam(shadow_server.parameters())
criterion = nn.CrossEntropyLoss()
device = 'cpu'
batch=64
shadow_server.train()
print(1)
softmax = nn.Softmax(dim=1)
for epoch in range(num_epoch):
    total_loss = 0
    for x,y in train_loader:
        y_label = softmax(serverSqueezenet(clientSqueezenet(x)).detach())
        y_shadow = shadow_server(x)
        
        optimizer.zero_grad()
        loss = criterion(y_shadow, y_label)
        total_loss += loss
        loss.backward()
        optimizer.step()
    print(f"epoch {(epoch+1)}: loss {total_loss/batch}")
print("done")

1
epoch 1: loss 20.908632278442383
epoch 2: loss 20.52694320678711
epoch 3: loss 20.442623138427734
epoch 4: loss 20.389854431152344
epoch 5: loss 20.37710189819336
epoch 6: loss 20.339073181152344
epoch 7: loss 20.351835250854492
epoch 8: loss 20.36783218383789
epoch 9: loss 20.342254638671875
epoch 10: loss 20.435396194458008
epoch 11: loss 20.41314697265625
epoch 12: loss 20.415699005126953
epoch 13: loss 20.373031616210938
epoch 14: loss 20.408363342285156
epoch 15: loss 20.396995544433594
epoch 16: loss 20.40045166015625
epoch 17: loss 20.35091781616211
epoch 18: loss 20.341148376464844
epoch 19: loss 20.358705520629883
epoch 20: loss 20.429418563842773
done


In [15]:
shadow_server.eval()
acc, tot = 0, 0
for x, y in test_loader:
    y_pred = shadow_server(x)
    acc += (y==y_pred.argmax(1)).sum()
    tot += len(y)
print(f"test acc : {acc}/{tot}")

test acc : 1021/10000


In [16]:
clientMobilenet = ClientMobileNet()
serverMobilenet = ServerMobileNet()
clientMobilenet.load_state_dict(torch.load('./assets/mobilenet_digit5_c1.pth'))
serverMobilenet.load_state_dict(torch.load('./assets/mobilenet_digit5_server.pth'))

<All keys matched successfully>

In [17]:
shadow_client = ClientMobileNet()
shadow_client.load_state_dict(torch.load('./assets/mobilenet_digit5_c1.pth'))
shadow_server = ShadowSqueezeNet()
mobile_squeeze = shadow_server.load_front_model(shadow_client)

In [18]:
# train 8000
indices = range(8000,16000)
mnist_train_loader = torch.utils.data.DataLoader(Subset(train_mnist, indices), batch_size=64, shuffle=False)
mnist_test_loader = torch.utils.data.DataLoader(test_mnist, batch_size=64, shuffle=False)

train_usps = torchvision.datasets.USPS(root='./data' + 'usps_data', train=True, download=True, transform=transform_to_rgb)
test_usps = torchvision.datasets.USPS(root='./data' + 'usps_data', train=False, download=True, transform=transform_to_rgb)

usps_train_loader = torch.utils.data.DataLoader(train_usps, batch_size=64, shuffle=False)
usps_test_loader = torch.utils.data.DataLoader(test_usps, batch_size=64, shuffle=False)

In [19]:
num_epoch = 20
optimizer = optim.Adam(shadow_server.parameters())
criterion = nn.CrossEntropyLoss()
device = 'cpu'
batch=64
shadow_server.train()
print(1)
softmax = nn.Softmax(dim=1)
for epoch in range(num_epoch):
    total_loss = 0
    for x,y in usps_train_loader:
        y_label = softmax(serverMobilenet(clientMobilenet(x)).detach())
        y_shadow = shadow_server(x)
        optimizer.zero_grad()
        loss = criterion(y_shadow, y_label)
        total_loss += loss
        loss.backward()
        optimizer.step()
    print(f"epoch {(epoch+1)}: loss {total_loss/batch}")
print("done")

1
epoch 1: loss 3.7576987743377686
epoch 2: loss 2.9482035636901855
epoch 3: loss 2.3557634353637695
epoch 4: loss 1.916349172592163
epoch 5: loss 1.7304333448410034
epoch 6: loss 1.6401710510253906
epoch 7: loss 1.5729318857192993
epoch 8: loss 1.5377154350280762
epoch 9: loss 1.5021400451660156
epoch 10: loss 1.5350316762924194
epoch 11: loss 1.5034326314926147
epoch 12: loss 1.4694615602493286
epoch 13: loss 1.4624309539794922
epoch 14: loss 1.449779748916626
epoch 15: loss 1.4303733110427856
epoch 16: loss 1.4309992790222168
epoch 17: loss 1.3858239650726318
epoch 18: loss 1.3820680379867554
epoch 19: loss 1.3882529735565186
epoch 20: loss 1.346755027770996
done


In [20]:
shadow_server.eval()
acc, tot = 0, 0
for x, y in usps_test_loader:
    y_pred = shadow_server(x)
    acc += (y==y_pred.argmax(1)).sum()
    tot += len(y)
print(f"test acc : {acc}/{tot}")

test acc : 1522/2007


In [21]:
torch.save(shadow_server, './mobile_squeeze_usps_co.pth')

In [22]:
shadow_client = ClientMobileNet()
shadow_client.load_state_dict(torch.load('./assets/mobilenet_digit5_c1.pth'))
shadow_server = ShadowSqueezeNet()
mobile_squeeze = shadow_server.load_front_model(shadow_client)

num_epoch = 20
optimizer = optim.Adam(shadow_server.parameters())
criterion = nn.CrossEntropyLoss()
device = 'cpu'
batch=64
shadow_server.train()
print(1)
softmax = nn.Softmax(dim=1)
for epoch in range(num_epoch):
    total_loss = 0
    for x,y in mnist_train_loader:
        y_label = softmax(serverMobilenet(clientMobilenet(x)).detach())
        y_shadow = shadow_server(x)
        optimizer.zero_grad()
        loss = criterion(y_shadow, y_label)
        total_loss += loss
        loss.backward()
        optimizer.step()
    print(f"epoch {(epoch+1)}: loss {total_loss/batch}")
print("done")

1
epoch 1: loss 4.313322067260742
epoch 2: loss 3.7973196506500244
epoch 3: loss 3.6821656227111816
epoch 4: loss 3.6508772373199463
epoch 5: loss 3.647822618484497
epoch 6: loss 3.6435370445251465
epoch 7: loss 3.6251089572906494
epoch 8: loss 3.6206424236297607
epoch 9: loss 3.6111395359039307
epoch 10: loss 3.612455368041992
epoch 11: loss 3.6165082454681396
epoch 12: loss 3.609854221343994
epoch 13: loss 3.6116552352905273
epoch 14: loss 3.6160542964935303
epoch 15: loss 3.6021130084991455
epoch 16: loss 3.7416164875030518
epoch 17: loss 3.633845090866089
epoch 18: loss 3.636012315750122
epoch 19: loss 3.6149606704711914
epoch 20: loss 3.6080105304718018
done


In [ ]:
shadow_server.eval()
acc, tot = 0, 0
for x, y in mnist_valid_loader:
    y_pred = shadow_server(x)
    acc += (y==y_pred.argmax(1)).sum()
    tot += len(y)
print(f"test acc : {acc}/{tot}")

In [ ]:
torch.save(shadow_server, './mobile_squeeze_mnist_co.pth')